In [1]:
import torch
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib.pyplot as plt

from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import soundfile
import librosa

In [2]:
processor = AutoProcessor.from_pretrained("openai/whisper-tiny")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-tiny")

c:\Users\Administrator\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 384)
      (layers): ModuleList(
        (0-3): 4 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=384, out_features=384, bias=False)
            (v_proj): Linear(in_features=384, out_features=384, bias=True)
            (q_proj): Linear(in_features=384, out_features=384, bias=True)
            (out_proj): Linear(in_features=384, out_features=384, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (fin

In [6]:
def predict_speech_to_text(audio_dir):
    #read the audio file
    audio = librosa.load(audio_dir, sr = 16000)
    print(audio)
    inputs = processor(audio[0], sampling_rate = 16000, return_tensors="pt", padding=True)
    print(inputs)

    # with torch.no_grad():
    #     logits = model(input_values=inputs.input_values.to(device)).logits

    # pred_ids = torch.argmax(logits, dim=-1)
    # prediction = processor.batch_decode(pred_ids)

    # return prediction

In [7]:
predict_speech_to_text("data/Actor_01/03-01-01-01-01-01-01.wav")

(array([ 1.4078640e-07, -1.9284332e-07,  2.5121108e-07, ...,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00], dtype=float32), 16000)


: 